In [ ]:
#1/5 Changes: Updated to pySBD
#This will generate 100 fragments with keywords
#change to data/

In [ ]:
import os, re
import pandas as pd
import numpy as np
import time


import medspacy, spacy, pysbd
from medspacy.ner import TargetRule 
from medspacy.visualization import visualize_ent
#from medspacy.sentence_splitting import PyRuSHSentencizer
from pysbd.utils import PySBDFactory
from keywords import find_keywords2

from matplotlib import pyplot as plt


In [ ]:
raw = pd.read_csv('data-raw/NOTEEVENTS.csv', nrows = 500000, low_memory = False)

# Lowercase the column names for easier typing.
raw.columns = raw.columns.str.lower()

print("Dataframe shape:", raw.shape)
print("Dataframe columns:", raw.columns)

In [ ]:
raw1=raw.drop_duplicates(subset=['subject_id','hadm_id', 'chartdate', 'category', 'description', 'cgid'], keep='last')
print("Dropping duplicates, Dataframe shape:", raw1.shape)

In [ ]:
drop=['Echo', 'ECG', 'Case Management ', 'Social Work', 'Pharmacy']
df=raw1[~raw1['category'].isin(drop)]
print("Dropping note types, Dataframe shape:", df.shape)

In [ ]:
all_keywords=[]
neg=pd.read_csv('data/negative_keywords.csv').iloc[::,0]
pos=pd.read_csv('data/positive_keywords.csv').iloc[::,0]

all_keywords.append(neg.apply(lambda x: x.strip()).tolist())
all_keywords.append(pos.apply(lambda x: x.strip()).tolist())

keywords=[i.lower() for i in all_keywords for i in i]

neg_keywords=neg.apply(lambda x: x.strip().lower()).tolist()
pos_keywords=pos.apply(lambda x: x.strip().lower()).tolist()

In [ ]:
#sentencizer=PyRuSHSentencizer()


nlp = medspacy.load(disable=['medspacy_pyrush', 'medspacy_target_matcher', 'medspacy_context'])
print(nlp.pipe_names)
nlp.add_pipe('medspacy_pysbd')
print(nlp.pipe_names)


In [ ]:
start=time.time()
sections=[]

##Change range to use all notes##
for i in range(0,min(500, df.text.shape[0])):
    doc=nlp(df.text.iloc[i])
    for sent in doc.sents:
        fragment=sent.text.strip().replace('\n',' ').replace('  ','')
        
        #Filters out short and long fragments
        if 5<len(fragment.split(' '))<500:
            sections.append(fragment)
end=time.time()
print(end-start)

In [ ]:
sentences=pd.DataFrame(sections, columns=['text'])
print(sentences.shape)

In [ ]:
def keyword_score(keyword_list, neg_keywords, pos_keywords):
    pos_score=len([i for i in keyword_list if i in pos_keywords])
    neg_score=len([i for i in keyword_list if i in neg_keywords])
    if neg_score+pos_score>0:
        return neg_score/(neg_score+pos_score)

In [ ]:
sentences['keywords']=sentences.text.map(lambda x: list(find_keywords2(x, keywords)))
sentences['keyword_presence']=sentences.keywords.map(lambda x: True if len(x)>0 else False)
sentences['keyword_score']=sentences.keywords.map(lambda x: keyword_score(x, neg_keywords, pos_keywords))

In [ ]:
print('starting number of notes', min(500, df.shape[0]))
print('total number of sentence fragments', sentences.shape[0])
print('number of sentences with keywords', sentences.keyword_presence.sum())

In [ ]:
relevant_fragments=sentences[sentences.keyword_presence==1].text
hundred_fragments=relevant_fragments.sample(n=100, random_state=1)


In [ ]:
hundred_fragments.iloc[75]

In [ ]:
hundred_fragments.to_csv('data/hundred_fragments.csv', index=False)

In [ ]:
#test
text=df.text.iloc[100]

In [ ]:
#Visualizing all sections (to see if they make sense)
#nlp = medspacy.load(enable=["sectionizer"])
#print(nlp.pipe_names)

doc=nlp(text)
#visualize_ent(doc)

In [ ]:
final_fragments=[]
for sent in doc.sents:
    fragment=sent.text.strip().replace('\n',' ').replace('   ',' ')
    if 5<len(fragment.split(' '))<500:
        final_fragments.append(fragment)


for frag in final_fragments[0:30]:
    print(frag)
    print('----'*10)

In [ ]:
final_fragments=[]
for sent in doc_pysbd.sents:
    fragment=sent.text.strip().replace('\n',' ').replace('  ','')
    if len(fragment)>10: final_fragments.append(fragment)


for frag in final_fragments:
    print(frag)
    print('----'*10)